# MCP Server using Fast API

In [ ]:
# Project: Voice-Based Emotional Wellness Journal (LLM + MCP)

# Directory Structure:
# voice_wellness_journal/
# ├── backend/
# │   ├── main.py              # FastAPI app entrypoint
# │   ├── audio_utils.py       # Audio saving and preprocessing
# │   ├── whisper_transcriber.py # WhisperX transcription
# │   ├── emotion_analyzer.py  # Emotion detection using LLM
# │   └── journal_generator.py # Journal prompts and suggestions
# ├── mcp/
# │   ├── mcp_server.py        # MCP server with tools and memory
# │   └── tools.py             # Registered MCP tools
# ├── frontend/ (React app, not in this code file)
# └── requirements.txt

# ------------------------------ backend/main.py ------------------------------
from fastapi import FastAPI, File, UploadFile
from backend.audio_utils import save_audio
from backend.whisper_transcriber import transcribe_audio
from mcp.mcp_server import run_mcp_agent

app = FastAPI()

@app.post("/upload-audio")
async def upload_audio(file: UploadFile = File(...)):
    path = save_audio(file)
    transcript = transcribe_audio(path)
    response = run_mcp_agent(session_id="user1", user_input=transcript)
    return {"transcript": transcript, "analysis": response}


# -------------------------- backend/audio_utils.py --------------------------
import os
from pathlib import Path
from fastapi import UploadFile

def save_audio(file: UploadFile) -> str:
    upload_path = Path("uploads")
    upload_path.mkdir(exist_ok=True)
    file_location = upload_path / file.filename
    with open(file_location, "wb") as buffer:
        buffer.write(file.file.read())
    return str(file_location)


# --------------------- backend/whisper_transcriber.py ----------------------
import whisperx

def transcribe_audio(path: str) -> str:
    model = whisperx.load_model("medium", device="cpu")
    audio = whisperx.load_audio(path)
    result = model.transcribe(audio, batch_size=8)
    return result["text"]


# --------------------- backend/emotion_analyzer.py -------------------------
from openai import OpenAI

client = OpenAI()

def analyze_emotion(text: str) -> str:
    prompt = f"""
    Analyze the emotion expressed in the following journal entry. Respond with a single word or short phrase (e.g., happy, anxious, content, frustrated).

    Journal Entry:
    {text}
    """
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()


# ------------------ backend/journal_generator.py --------------------------
from openai import OpenAI

client = OpenAI()

def generate_journal_prompt(emotion: str) -> str:
    prompt = f"""
    The user expressed the emotion: {emotion}.
    Suggest a reflective journal prompt to help process this feeling.
    """
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()


# ------------------------------ mcp/tools.py ------------------------------
from backend.emotion_analyzer import analyze_emotion
from backend.journal_generator import generate_journal_prompt
from mcp.mcp_utils import tool

@tool("analyze_emotion")
def emotion_tool(context, transcript: str):
    return analyze_emotion(transcript)

@tool("generate_prompt")
def prompt_tool(context, emotion: str):
    return generate_journal_prompt(emotion)


# -------------------------- mcp/mcp_server.py -----------------------------
from mcp.agent import MCPAgent
from mcp.memory import MemoryBuffer
from mcp.tools import emotion_tool, prompt_tool

memory = MemoryBuffer()
agent = MCPAgent(tools=[emotion_tool, prompt_tool], memory=memory)

def run_mcp_agent(session_id: str, user_input: str):
    return agent.run(session_id=session_id, user_input=user_input)


# -------------------------- requirements.txt ------------------------------
fastapi
uvicorn
openai
whisperx
torch
tqdm
mcp-protocol
python-dotenv

# ----------------------------- END OF FILE --------------------------------

# ✅ Frontend idea (not included here):
# - Mic recorder (React): Save and send audio
# - Display journal entry, detected emotion, suggested prompt
# - Add calendar view to show entries by date


In [ ]:
# mcp/client.py

import requests
from typing import Optional

class MCPClient:
    def __init__(self, api_url: str):
        """
        Initialize the MCP Client with the URL of the FastAPI or MCP server.
        Example: http://localhost:8000
        """
        self.api_url = api_url.rstrip("/")

    def send_audio(self, audio_path: str) -> dict:
        """
        Sends an audio file to the backend for processing.
        :param audio_path: Path to the local audio file (.wav/.mp3)
        :return: Dictionary with transcript and analysis
        """
        with open(audio_path, "rb") as f:
            files = {"file": (audio_path, f, "audio/mpeg")}
            response = requests.post(f"{self.api_url}/upload-audio", files=files)
            response.raise_for_status()
            return response.json()

    def analyze_text(self, session_id: str, text: str) -> dict:
        """
        Directly sends text to the MCP agent if transcription is already done.
        :param session_id: Identifier for session memory tracking
        :param text: Journal or spoken content
        :return: Processed output
        """
        payload = {
            "session_id": session_id,
            "user_input": text
        }
        response = requests.post(f"{self.api_url}/mcp-analyze", json=payload)
        response.raise_for_status()
        return response.json()



In [ ]:
from fastapi import Request
from pydantic import BaseModel

class TextRequest(BaseModel):
    session_id: str
    user_input: str

@app.post("/mcp-analyze")
async def analyze_with_text(payload: TextRequest):
    result = run_mcp_agent(session_id=payload.session_id, user_input=payload.user_input)
    return result


In [ ]:
# test_client.py
from mcp.client import MCPClient

client = MCPClient(api_url="http://localhost:8000")

# Example 1: Using audio file
result = client.send_audio("sample_recording.wav")
print("Transcript:", result["transcript"])
print("Analysis:", result["analysis"])

# Example 2: Direct text analysis
text = "I'm feeling really anxious about my exams."
text_response = client.analyze_text(session_id="test123", text=text)
print("LLM Response:", text_response)
